In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import warnings
# warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import copy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
from scipy.special import expit

In [3]:
data = pd.read_csv('test_USA.csv')
X = data.iloc[:, :-1]  # Select all columns except the last one
y=data["GSNR_1"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train=np.array(X_train)
y_train=np.array(y_train)
y_train=y_train.reshape(-1,1)

In [5]:
#========normalization==========
#scaler = MinMaxScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
L_scaler=StandardScaler()
y_train=L_scaler.fit_transform(y_train)
print("X_train",X_train.shape)
print("y_train",y_train.shape)
print("X_test",X_test.shape)
print("y_test",y_test.shape)
X_train

X_train (26400, 305)
y_train (26400, 1)
X_test (6600, 305)
y_test (6600,)


array([[-0.84269354,  1.22122761,  1.22496227, ..., -0.63240593,
        -0.13960499, -0.09659711],
       [-0.84269354,  1.22557692, -0.83031147, ..., -0.63240593,
         0.18780349,  0.16523775],
       [-0.84269354, -0.83164554,  1.22931666, ..., -0.63240593,
        -0.13960499, -0.09659711],
       ...,
       [ 1.12868526,  1.14511473,  1.15093758, ..., -0.63240593,
         1.23551062,  1.20810742],
       [-0.84269354, -0.83164554, -0.83031147, ..., -0.63240593,
        -0.27056838, -0.27809368],
       [-0.84269354, -0.6296202 , -0.83031147, ...,  3.60593093,
        -1.38375721, -1.34580137]])

In [6]:
def initialize(n_x,n_h,n_y):
    np.random.seed(2)
    
    W1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    
    parameters={"W1":W1,
               "b1":b1,
               "W2":W2,
               "b2":b2}
    return parameters

In [7]:
def ReLU(x):
    return x * (x > 0)
def ReLU_derivative(x):
    return (x>=0)

In [8]:
def leaky_relu(x, alpha=0.01):
    return np.maximum(alpha * x, x)

In [8]:
def forward_propagation(X,parameters):
    
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    
    
    
    
    Z1=np.dot(W1,X.T)+b1
    #A1=np.tanh(Z1) 
    #A1=1/(1+np.exp(-Z1))
    A1=ReLU(Z1)
    Z2=np.dot(W2,A1)+b2
    #A2=Z2
    A2=ReLU(Z2)
    #A2=leaky_relu(Z2)
    #A2=1/(1+np.exp(-Z2))
    
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    #print("Z1",Z1.shape)
    #print("A1",A1.shape)
    #print("Z2",Z2.shape)
    #print("A2",A2.shape)


    return A2, cache

In [14]:
def compute_cost(A2, Y):
    
    
    
    m=Y.shape[0]
    
    #cost=(-1/m)*(np.dot(Y, np.log(A2+0.000001).T) + np.dot((1-Y), np.log((1-A2)+0.000001).T))
    cost=np.sum((A2-Y)**2)/(m)
    cost = float(np.squeeze(cost)) 
    return cost

In [15]:
def back_propagation(X,Y,parameters,cache):
    m=X.shape[0]
    
    
    
    W1=parameters["W1"]
    W2=parameters["W2"]
    A1=cache["A1"]
    A2=cache["A2"]
    Z1=cache["Z1"]
    
    #cros entropy loss back prop
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2,A1.T)
    db2=(1/m)*(np.sum(dZ2,axis=1,keepdims=True))
    #dZ1=np.dot(W2.T,dZ2) *(1 - np.power(A1, 2))
    #dZ1=np.dot(W2.T,dZ2) *(A1*(1-A1))
    dZ1=np.dot(W2.T,dZ2) *ReLU_derivative(Z1)   
    dW1=(1/m) *(np.dot(dZ1,X))
    db1=(1/m)*(np.sum(dZ1,axis=1,keepdims=True))
    #============================
    
    grads={"dW2":dW2,
          "db2":db2,
          "dW1":dW1,
          "db1":db1}
#     print("dw1",dW1.shape)
#     print("dW2",dW2.shape)
    return grads

In [16]:
def update_parameters(parameters,grads,learning_rate):
    
    W1=parameters["W1"]
    W2=parameters["W2"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    
    
    
    dW1=grads["dW1"]
    dW2=grads["dW2"]
    db2=grads["db2"]
    db1=grads["db1"]
    
    
    W1=W1-learning_rate*dW1
    W2=W2-learning_rate*dW2
    b1=b1-learning_rate*db1
    b2=b2-learning_rate*db2
    
    
    parameters={"W1":W1,
               "W2":W2,
               "b1":b1,
               "b2":b2}
    return parameters

In [17]:
def train_model(X,Y, n_h, num_iterations = 10000, print_cost=False):
    n_x=X.shape[1]
    n_y=1
    
    
    parameters=initialize(n_x,n_h,n_y)
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    
    
    
    for i in range(0,num_iterations):
        A2, cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y)
        
        
        grads = back_propagation(X,Y,parameters,cache)
        
        parameters = update_parameters(parameters, grads, 0.001)#0.001
        
        
        
        if (i%100==0) :
            print("cost after",i,"iteration is",cost)
            
            
    return parameters,cost

In [19]:
parameters,cost=train_model(X_train,y_train,8 , num_iterations = 1000, print_cost=True)
print("The cost after 1000 iterations is ",cost)

cost after 0 iteration is 514.8677847334243
cost after 100 iteration is 317.75608425328574
cost after 200 iteration is 191.76463750204394
cost after 300 iteration is 153.41401682619548
cost after 400 iteration is 148.98892747705187
cost after 500 iteration is 147.10481126589715
cost after 600 iteration is 145.8180571316407
cost after 700 iteration is 144.78974917443836
cost after 800 iteration is 143.9307303813035
cost after 900 iteration is 143.19925043830065
The cost after 1000 iterations is  142.56865300706886
